<a href="https://colab.research.google.com/github/Ariiiff/NLP_Practices/blob/main/bengali_hate_speech_detection_using_ml_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
import itertools
from wordcloud import WordCloud
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM, SpatialDropout1D, Embedding
from keras.utils import to_categorical
from joblib import dump, load

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Reading the test dataset
test_df = pd.read_csv('/content/gdrive/MyDrive/NLP/Project/Bangali-hate-speech-detection/Datasets/test.csv')

# Reading the train dataset
train_df = pd.read_csv('/content/gdrive/MyDrive/NLP/Project/Bangali-hate-speech-detection/Datasets/train.csv')


# Reading the test dataset
val_df = pd.read_csv('/content/gdrive/MyDrive/NLP/Project/Bangali-hate-speech-detection/Datasets/validate.csv')

In [4]:
dfs = [train_df, test_df, val_df]
#merge all DataFrames into one
final_df = pd.concat(dfs)

In [5]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4500 entries, 0 to 899
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4500 non-null   object
 1   label   4500 non-null   object
dtypes: object(2)
memory usage: 105.5+ KB


In [6]:
final_df

,text,label
0,ধর্মীয় বইগুলোকে বলা হচ্ছে উগ্রবাদী বই।,Religious
1,দাদা! নৌকা কিন্তু চলে না। এমাসে কম দিয়েছেন ১৯ ...,Geopolitical
2,"দম্ভ ও অহংকার পতনের মূল, জাতি দম্ভ দেখেছে এখন ...",Geopolitical
3,বিএনপি জামাতের আগুন সন্ত্রাস। রাজধানীতে বাসে আ...,Geopolitical
4,পুলিশকে ২০ দলের হুশিয়ারী বেশি বাড়াবাড়ি করলে তা...,Geopolitical
...,...,...
895,"নায়ক যখন অনেক গরীব, কিন্তু নাইকার বাবা অনেক ধনী।",Neutral
896,"পৃথিবীতে একটাই সমস্যা, সেটা হলো বউ। কেউ নিয়ে আ...",Neutral
897,"আমাদের কাহিনীর মধ্যে ভুত, আমার দিকে একটু ফোকাস...",Neutral
898,"কয়েক মিনিট পর পর রবি কোম্পানি, তোর কি ইন্টারনে...",Neutral


In [7]:
pd.unique(final_df['label'])

array(['Religious', 'Geopolitical', 'Neutral', 'Personal', 'Political'],
      dtype=object)

In [8]:
def label_classification(value) :
  if value == 'Neutral':
        return 0
  if value == "Religious":
        return 1
  elif value == 'Geopolitical':
        return 2
  elif value == 'Personal':
        return 3
  elif value == 'Political':
        return 4

In [9]:
final_df['class'] = final_df['label'].map(label_classification)

In [10]:
final_df

,text,label,class
0,ধর্মীয় বইগুলোকে বলা হচ্ছে উগ্রবাদী বই।,Religious,1
1,দাদা! নৌকা কিন্তু চলে না। এমাসে কম দিয়েছেন ১৯ ...,Geopolitical,2
2,"দম্ভ ও অহংকার পতনের মূল, জাতি দম্ভ দেখেছে এখন ...",Geopolitical,2
3,বিএনপি জামাতের আগুন সন্ত্রাস। রাজধানীতে বাসে আ...,Geopolitical,2
4,পুলিশকে ২০ দলের হুশিয়ারী বেশি বাড়াবাড়ি করলে তা...,Geopolitical,2
...,...,...,...
895,"নায়ক যখন অনেক গরীব, কিন্তু নাইকার বাবা অনেক ধনী।",Neutral,0
896,"পৃথিবীতে একটাই সমস্যা, সেটা হলো বউ। কেউ নিয়ে আ...",Neutral,0
897,"আমাদের কাহিনীর মধ্যে ভুত, আমার দিকে একটু ফোকাস...",Neutral,0
898,"কয়েক মিনিট পর পর রবি কোম্পানি, তোর কি ইন্টারনে...",Neutral,0


In [11]:
new_df = final_df.dropna()

In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4500 entries, 0 to 899
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4500 non-null   object
 1   label   4500 non-null   object
 2   class   4500 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 140.6+ KB


In [13]:
#data cleaning
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [14]:
clean_texts = []
for txt in new_df["text"] :
  clean_texts.append(preprocessing(txt))

In [15]:
data1 = pd.read_excel('/content/gdrive/MyDrive/NLP/Project/Bangali-hate-speech-detection/Datasets/stopwords_bangla.xlsx')
stopwords = data1['words'].tolist()

In [16]:
def stopwordRemoval(text):
    x=str(text)
    l=x.split()

    stm=[elem for elem in l if elem not in stopwords]

    out=' '.join(stm)

    return str(out)

In [17]:
new_clean_texts = []
for txt in clean_texts :
  new_clean_texts.append(stopwordRemoval(txt))

In [18]:
new_clean_texts[: 10]

['ধর্মীয় বইগুলোকে উগ্রবাদী বই',
 'দাদা নৌকা না এমাসে কিউসেক পানি নদীতে পানির দরকার বন্ধুত্বে জোয়ার বইছে',
 'দম্ভ অহংকার পতনের মূল জাতি দম্ভ দেখেছে পতন',
 'বিএনপি জামাতের আগুন সন্ত্রাস রাজধানীতে বাসে আগুন দেয়ার ঘাঁনয় বিএনপি জামাতের আটক',
 'পুলিশকে দলের হুশিয়ারী বাড়াবাড়ি বাড়ীতে বাড়ীতে আক্রমন',
 'চুইংগাম চিবানোর সময় পাকস্থলিও কনফিউজড যায় ঘন্টা চিবুচ্ছে না',
 'পি আসার রাগের মাথায় নুনু তে কয়ডা থাপ্পর মারলাম',
 'গ্রাম এসেই চাচাতো ভাই চুরি আব্বু চাচাতো ভাই',
 'আনুশকা দিহানের গোপন ভিডিও মামুনুল হক বললো দেখুন ভিডিওতে',
 'ব্যাচেলর ছেলেকে বাসার ছাদে বালতি ভেজা কাপড় শুকাতে দেখার']

In [19]:
new_df['clean_text'] = new_clean_texts

In [53]:
new_df[20:30]

,text,label,class,clean_text
20,নারী কিংবা পুরুষ যে কোনো অপরাধী ধরা পড়লে কাদের...,Neutral,0,নারী পুরুষ অপরাধী পড়লে কাদের ভাইয়ের সাথে ছবি প...
21,"মা, আমি সবাইকে চিনতে পারছি।",Neutral,0,মা সবাইকে চিনতে পারছি
22,তারপর আলগা ভাব নিয়ে ঐ মেয়েটার সামনে যেতে লাগলাম।,Neutral,0,আলগা ভাব মেয়েটার লাগলাম
23,তাতে আমার বাল ছেড়া গেলো।,Personal,3,বাল ছেড়া গেলো
24,যখন মাঝ রাতে হিসি করতে উঠে দেখো তোমার সামনে সা...,Neutral,0,মাঝ রাতে হিসি উঠে দেখো সাদা শাড়ি পড়ে দাড়িয়ে
25,রাস্তায় হাটার সময় যখন কোনো সুন্দরী মেয়ে তাকিয়ে...,Neutral,0,রাস্তায় হাটার সময় সুন্দরী মেয়ে তাকিয়ে মুচকি হাসি
26,"হাওলাতের মায়রে চুঁদি, বাকীর মায়রে চুঁদি, পীরিত...",Personal,3,হাওলাতের মায়রে চুঁদি বাকীর মায়রে চুঁদি পীরিতের...
27,মেলায় হারিয়ে যাওয়া দুই গাঁজাখোর ভাই।,Personal,3,মেলায় হারিয়ে গাঁজাখোর ভাই
28,ধর্ম যার যার উৎসব সবার এটা ইসলাম বিরোধী উক্তি।...,Religious,1,ধর্ম উৎসব ইসলাম বিরোধী ধর্ম উৎসব
29,চট্টগ্রামের প্রাণকেন্দ্র আগ্রাবাদে দেশদ্রোহী দ...,Personal,3,চট্টগ্রামের প্রাণকেন্দ্র আগ্রাবাদে দেশদ্রোহী দ...


In [58]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(new_df['clean_text'], new_df['class'], test_size= 0.2, random_state=42)

In [22]:
xtrain.shape, xtest.shape

((3600,), (900,))

In [23]:
ytrain.shape, ytest.shape

((3600,), (900,))

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

In [25]:
model = make_pipeline(CountVectorizer(), MultinomialNB())

In [26]:
model.fit(xtrain,ytrain)

ypred = model.predict(xtest)

In [27]:
classification_rep = classification_report(ytest, ypred)

print(classification_rep)

              precision    recall  f1-score   support

           0       0.63      0.84      0.72       409
           1       0.88      0.13      0.23        54
           2       0.59      0.51      0.54       126
           3       0.51      0.43      0.47       281
           4       0.33      0.03      0.06        30

    accuracy                           0.60       900
   macro avg       0.59      0.39      0.40       900
weighted avg       0.59      0.60      0.57       900



In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

xtrain, xtest, ytrain, ytest = train_test_split(new_df['clean_text'], new_df['class'], test_size= 0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(xtrain)
X_test_tfidf = tfidf_vectorizer.transform(xtest)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, ytrain)

RandomForestClassifier(random_state=42)

In [29]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = rf_classifier.predict(X_test_tfidf)
accuracy = accuracy_score(ytest, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(ytest, y_pred))

Accuracy: 0.57
              precision    recall  f1-score   support

           0       0.61      0.80      0.69       409
           1       0.78      0.26      0.39        54
           2       0.51      0.49      0.50       126
           3       0.51      0.40      0.45       281
           4       0.20      0.03      0.06        30

    accuracy                           0.57       900
   macro avg       0.52      0.40      0.42       900
weighted avg       0.56      0.57      0.55       900



In [57]:
text ="ধর্মীয় বইগুলোকে বলা হচ্ছে উগ্রবাদী বই।	"

result = preprocessing(text)


res = (stopwordRemoval(result))

new_text_tfidf = tfidf_vectorizer.transform([res])
res_predicted = rf_classifier.predict(new_text_tfidf)

if res_predicted == 0:
      print('Neutral')
if res_predicted == 1:
      print("Religious")
elif res_predicted == 2:
      print('Geopolitical')
elif res_predicted == 3:
      print('Personal')
elif res_predicted == 4:
      print('Political')

print(res_predicted)

Religious
[1]


In [41]:
text ="কুত্তার বাচ্চা পাপন"

result = preprocessing(text)


res = (stopwordRemoval(result))

new_text_tfidf = tfidf_vectorizer.transform([res])
res_predicted = rf_classifier.predict(new_text_tfidf)

if res_predicted == 0:
      print('Neutral')
if res_predicted == 1:
      print("Religious")
elif res_predicted == 2:
      print('Geopolitical')
elif res_predicted == 3:
      print('Personal')
elif res_predicted == 4:
      print('Political')

print(res_predicted)

Personal
[3]


In [48]:
text ="শেখ হাসিনা আছে বলেই করোনা প্রতিরোধ করতে পারছি"

result = preprocessing(text)


res = (stopwordRemoval(result))

new_text_tfidf = tfidf_vectorizer.transform([res])
res_predicted = rf_classifier.predict(new_text_tfidf)

if res_predicted == 0:
      print('Neutral')
if res_predicted == 1:
      print("Religious")
elif res_predicted == 2:
      print('Geopolitical')
elif res_predicted == 3:
      print('Personal')
elif res_predicted == 4:
      print('Political')

print(res_predicted)

Political
[4]
